In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

%store -r tx_level_data

pd.set_option('display.width', 200)  # Set a large width
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line wrapping



print(tx_level_data.head())

            timestamp                                          signature      type          source tx_status direction                                        sender                                      receiver                                  counterparty  Native SOL Amount token_address  token_amount  block_number    tx_fee                        program_id  PRE_BALANCE  POST_BALANCE SYMBOL TOKEN_NAME                                         MINT      sender_name    receiver_name counterparty_name     program_name wallet_entity_label
0 2025-04-06 12:12:19  6mpqrks4akyjez3jljjubhswnb96oijdfx1dgeznzebhxm...  TRANSFER  SYSTEM_PROGRAM   success  received  5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr  AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU  5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr       1.000000e-09          None           NaN   331670303.0  0.000006  11111111111111111111111111111111     0.017773      0.017773    SOL     Solana  So11111111111111111111111111111111111111111  Unknow

In [2]:
tx_level_data.columns.to_list()

['timestamp',
 'signature',
 'type',
 'source',
 'tx_status',
 'direction',
 'sender',
 'receiver',
 'counterparty',
 'Native SOL Amount',
 'token_address',
 'token_amount',
 'block_number',
 'tx_fee',
 'program_id',
 'PRE_BALANCE',
 'POST_BALANCE',
 'SYMBOL',
 'TOKEN_NAME',
 'MINT',
 'sender_name',
 'receiver_name',
 'counterparty_name',
 'program_name',
 'wallet_entity_label']

# Data Cleaning 

In [3]:
def clean_tx_data(df: pd.DataFrame) -> pd.DataFrame:
    # Drop duplicates
    df = df.drop_duplicates(subset=['signature'])

    # Fill missing wallet names and labels with 'Unknown'
    # Fill name fields with wallet addresses if 'Unknown Address'
    df['sender_name'] = np.where(
        df['sender_name'] == 'Unknown Address',
        df['sender'].str.slice(0, 6),
        df['sender_name']
    )
    df['receiver_name'] = np.where(
        df['receiver_name'] == 'Unknown Address',
        df['receiver'].str.slice(0, 6),
        df['receiver_name']
    )
    df['counterparty_name'] = np.where(
        df['counterparty_name'] == 'Unknown Address',
        df['counterparty'].str.slice(0, 6),
        df['counterparty_name']
    )
    
    # Fill missing wallet_entity_label with 'Unknown'
    df['wallet_entity_label'] = df['wallet_entity_label'].fillna('Unknown Address')
    
    # Convert timestamp to datetime object
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # 4. Filter for successful SOL transfers
    df = df[
        (df['tx_status'] == 'success') ]
    
    # 5. Remove duplicates based on signature
    df = df.drop_duplicates(subset=['signature'])
    
    # 6. Reset index
    df = df.reset_index(drop=True)


    # Ensure numerical columns are clean
    numeric_cols = ['Native SOL Amount', 'token_amount', 'tx_fee', 'PRE_BALANCE', 'POST_BALANCE']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    return df

# Transaction Flow Aggregation

In [4]:
def build_tx_graph(df: pd.DataFrame):
    nodes = {}
    edges = []

    for _, row in df.iterrows():
        sender = row['sender']
        receiver = row['receiver']
        pre_balance = row['PRE_BALANCE'] or 0
        post_balance = row['POST_BALANCE'] or 0
        net_change = post_balance - pre_balance
        native_amount = row['Native SOL Amount'] or 0

        # Add sender node
        if sender not in nodes:
            nodes[sender] = {
                "id": sender,
                "label": row['sender_name'],
                "entity": row['wallet_entity_label'],
                "type": "sender",
                "pre_balance": pre_balance,
                "post_balance": post_balance,
                "net_balance_change": net_change,
                "native_sol_amount": native_amount
            }
        else:
            # Accumulate net change and balances if already added
            nodes[sender]["pre_balance"] = min(nodes[sender]["pre_balance"], pre_balance)
            nodes[sender]["post_balance"] = max(nodes[sender]["post_balance"], post_balance)
            nodes[sender]["net_balance_change"] += net_change
            nodes[sender]["native_sol_amount"] += native_amount
            
        # Add receiver node
        if receiver not in nodes:
            nodes[receiver] = {
                "id": receiver,
                "label": row['receiver_name'],
                "entity": row['wallet_entity_label'],
                "type": "receiver",
                "pre_balance": pre_balance,
                "post_balance": post_balance,
                "net_balance_change": net_change,
                "native_sol_amount": native_amount
            }
        else:
            # Accumulate if already added
            nodes[receiver]["pre_balance"] = min(nodes[receiver]["pre_balance"], pre_balance)
            nodes[receiver]["post_balance"] = max(nodes[receiver]["post_balance"], post_balance)
            nodes[receiver]["net_balance_change"] += net_change
            nodes[receiver]["native_sol_amount"] += native_amount



        # Create edge
        edge = {
            "source": sender,
            "target": receiver,
            "amount": native_amount or row['token_amount'],
            "symbol": row['SYMBOL'],
            "token": row['TOKEN_NAME'],
            "timestamp": row['timestamp'].isoformat(),
            "tx_type": row['type'],
            "program": row['program_name'],
            "signature": row['signature']
        }
        edges.append(edge)

    return {
        "nodes": list(nodes.values()),
        "edges": edges
    }

In [5]:
# Example usage
if __name__ == "__main__":


    tx_level_data_clean = clean_tx_data(tx_level_data)
    tx_graph = build_tx_graph(tx_level_data_clean)

    # Export to JSON for frontend use
    import json
    with open("tx_graph.json", "w") as f:
        json.dump(tx_graph, f, indent=2, default=str)

    print("✅ Cleaned and graph data saved to tx_graph.json")

✅ Cleaned and graph data saved to tx_graph.json
